In [1]:
import pandas as pd
import numpy as np
from data_processing import *

In [ ]:
# load training and test batches
train_batches, test_batch = load_batches()

# choose airplanes label (0)
train_x, train_y = process_batch(combine_batches(train_batches), 0, 5000)
test_x, test_y = process_batch(test_batch, 0, 1000)

In [4]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import tensorflow as tf
from tensorflow.keras import Input, layers, Sequential, optimizers, losses, callbacks

In [5]:
# hyperparameter values
HP_NUM_UNITS_1 = hp.HParam('num_units_1', hp.Discrete([28, 32]))
HP_NUM_UNITS_2 = hp.HParam('num_units_2', hp.Discrete([48, 52]))
HP_NUM_UNITS_3 = hp.HParam('num_units_3', hp.Discrete([68, 72]))
HP_NUM_UNITS_4 = hp.HParam('num_units_4', hp.Discrete([100, 104]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.15, 0.2]))

METRIC_ACCURACY = 'accuracy'

In [6]:
# configure hyperparameters and accuracy metric
with tf.summary.create_file_writer('logs/hparam_tuning/1').as_default():
    hp.hparams_config(
        hparams=[
            HP_NUM_UNITS_1,
            HP_NUM_UNITS_2,
            HP_NUM_UNITS_3,
            HP_NUM_UNITS_4,
            HP_DROPOUT,
            ],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [7]:
def train_test_model(hparams):
    # augment input data randomly
    data_augmentation = Sequential(
    [
        layers.RandomFlip("horizontal",
        input_shape=(32,32,3)),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
    )

    model = Sequential(name="Image_Recognition_Model")
    model.add(Input(shape=(32,32,3,))) # Input layer

    model.add(data_augmentation)
    model.add(layers.Rescaling(scale=1./255, name="Normaliser")) # normalisation layer

    model.add(layers.Conv2D(hparams[HP_NUM_UNITS_1], 3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D())

    model.add(layers.Conv2D(hparams[HP_NUM_UNITS_2], 3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D())

    model.add(layers.Conv2D(hparams[HP_NUM_UNITS_3], 3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D())

    model.add(layers.Dropout(hparams[HP_DROPOUT]))

    model.add(layers.Flatten())

    model.add(layers.Dense(hparams[HP_NUM_UNITS_4], activation='relu'))
    model.add(layers.Dense(2))
    
    model.compile(
        optimizer='adam',
        loss=losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    model.fit(
        train_x,
        train_y,
        epochs=5,
        verbose=0,
    )

    _, accuracy = model.evaluate(test_x, test_y)

    return accuracy

In [8]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [9]:
session_num = 0

# loop through possible combinations of hyperparameter values
for num_units_1 in HP_NUM_UNITS_1.domain.values:
    for num_units_2 in HP_NUM_UNITS_2.domain.values:
        for num_units_3 in HP_NUM_UNITS_3.domain.values:
            for num_units_4 in HP_NUM_UNITS_4.domain.values:
                for dropout in HP_DROPOUT.domain.values:
                    hparams = {
                        HP_NUM_UNITS_1: num_units_1,
                        HP_NUM_UNITS_2: num_units_2,
                        HP_NUM_UNITS_3: num_units_3,
                        HP_NUM_UNITS_4: num_units_4,
                        HP_DROPOUT: dropout,
                    }
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
                    print({h.name: hparams[h] for h in hparams})
                    run('logs/hparam_tuning/1/' + run_name, hparams)
                    session_num += 1


--- Starting trial: run-0
{'num_units_1': 28, 'num_units_2': 48, 'num_units_3': 68, 'num_units_4': 100, 'dropout': 0.15}
63/63 [==============================] - 1s 10ms/step - loss: 0.3517 - accuracy: 0.8480
--- Starting trial: run-1
{'num_units_1': 28, 'num_units_2': 48, 'num_units_3': 68, 'num_units_4': 100, 'dropout': 0.2}
63/63 [==============================] - 1s 11ms/step - loss: 0.3824 - accuracy: 0.8295
--- Starting trial: run-2
{'num_units_1': 28, 'num_units_2': 48, 'num_units_3': 68, 'num_units_4': 104, 'dropout': 0.15}
63/63 [==============================] - 1s 10ms/step - loss: 0.3581 - accuracy: 0.8485
--- Starting trial: run-3
{'num_units_1': 28, 'num_units_2': 48, 'num_units_3': 68, 'num_units_4': 104, 'dropout': 0.2}
63/63 [==============================] - 1s 11ms/step - loss: 0.3665 - accuracy: 0.8540
--- Starting trial: run-4
{'num_units_1': 28, 'num_units_2': 48, 'num_units_3': 72, 'num_units_4': 100, 'dropout': 0.15}
63/63 [==============================] - 1s 1